In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import io
import zipfile
import pandas as pd
import requests

TODO
- make a function to access the zipped & nonzipped files? Becuase need to read in as request, then pass to zipfile or pandas.
- decide how many years to allow
- make a function to clean up column names

### Get and format imputs.

In [27]:
# The 5 user inputs will be

year = 2015
state_abbr = 'AL' 
table = 'B00001'
geo_unit = 'non_block' # data for block level only or all other geographies.
content_type = 'estimate' # retrieve estimates or margins of error.

In [28]:
# First run the `state_dict` cell below. 

# Tranform/format arguments

year = str(year) # todo: confirm this is needed

# standardize formatting
state_abbr = state_abbr.lower()

# Get geographic level of data to use in data import url

# **Laurel** I don't think I've seen code formatted like the if statments below. Usually they're inside of another function. Is this ok?

if geo_unit == 'block':
    url_geo = 'Tracts_Block_Groups_Only'
if geo_unit == 'non_block':
    url_geo = 'All_Geographies_Not_Tracts_Block_Groups'

# Get full state name. 
state_name = state_dict[state_abbr]

# Get sequence number
sequence_table_url = 'http://www2.census.gov/programs-surveys/acs/summary_file/{0}/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.xls'.format(year)
sequence_table_df = pd.read_excel(sequence_table_url, dtype=str)

# clean up column names
sequence_table_df.columns = sequence_table_df.columns.str.replace(' ', '_').str.lower()

# find the sequence number of the first occurance of the desired table and left pad it
sequence = '{:0>4}'.format(sequence_table_df['sequence_number'].loc[sequence_table_df['table_id'] == table].max())


### Import raw files

In [29]:
# raw data file

url = 'https://www2.census.gov/programs-surveys/acs/summary_file/{0}/data/5_year_seq_by_state/{1}/{2}/{3}5{4}{5}000.zip'.\
format(year, state_name, url_geo, year, state_abbr, sequence )

results = requests.get(url)

zipped = zipfile.ZipFile(io.BytesIO(results.content))

for value in content_type:
    if content_type == 'margin':
        raw_csv = zipped.extract('m{0}5{1}{2}000.txt'.format(year,state_abbr, sequence)) # saves in pwd, unless other is specified
        raw_df = pd.read_csv(raw_csv, dtype=str,header=None)
    elif content_type == 'estimate':
        raw_csv = zipped.extract('e{0}5{1}{2}000.txt'.format(year, state_abbr, sequence))
        raw_df = pd.read_csv(raw_csv, dtype=str,header=None)
    else:
        print('no file') # need this?

working: 
https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_seq_by_state/Alabama/All_Geographies_Not_Tracts_Block_Groups/20155al0001000.zip

In [ ]:
# Import header file

    # 1) download zip file
    # 2) extract the file with format 'Seq85.xls'. Will need to drop leading zeros from sequence or make two variables, padded_sequence for the 
    # url above and unpadded number for this file extraction.

In [ ]:
# Import geographic location name file

### Append raw data and header files.

In [171]:
# Change raw_df column names to align with header** column names
raw_df.columns = header_df.columns

### Merge geographic location file.

In [31]:
state_dict = {
'ak':'alaska',
'al':'alabama',
'ar':'arkansas',
'az':'arizona',
'ca':'california',
'co':'colorado',
'ct':'connecticut',
'dc':'districtofcolumbia',
'de':'delaware',
'fl':'florida',
'ga':'georgia',
'gu':'guam',
'hi':'hawaii',
'ia':'iowa',
'id':'idaho',
'il':'illinois',
'in':'indiana',
'ks':'kansas',
'ky':'kentucky',
'la':'louisiana',
'ma':'massachusetts',
'md':'maryland',
'me':'maine',
'mi':'michigan',
'mn':'minnesota',
'mo':'missouri',
'ms':'mississippi',
'mt':'montana',
'na':'national',
'nc':'northcarolina',
'nd':'northdakota',
'ne':'nebraska',
'nh':'newhampshire',
'nj':'newjersey',
'nm':'newmexico',
'nv':'nevada',
'ny':'newyork',
'oh':'ohio',
'ok':'oklahoma',
'or':'oregon',
'pa':'pennsylvania',
'pr':'puertorico',
'ri':'rhodeisland',
'sc':'southcarolina',
'sd':'southdakota',
'tn':'tennessee',
'tx':'texas',
'ut':'utah',
'va':'virginia',
'vi':'virginislands',
'vt':'vermont',
'wa':'washington',
'wi':'wisconsin',
'wv':'westvirginia',
'wy':'wyoming' 
}